## Evaluating a vectorial function on CPU and GPU

### CPU: plain and numpy

In [7]:
import numpy as np
from numba import njit, jit

# Python plain implementation w/ numba 
@njit
def grade2_vector(x, y, a, b, c):
    z = np.zeros(x.size)
    for i in range(x.size):
        z[i] = a*x[i]*x[i] + b*y[i] + c
    return z

# Numpy ufunc
def grade2_ufunc(x, y, a, b, c):
    return a*x**2 + b*y + c

# size of the vectors
size = 5_000_000

# allocating and populating the vectors
a_cpu = np.random.rand(size)
b_cpu = np.random.rand(size)
c_cpu = np.zeros(size)

a = 3.5
b = 2.8
c = 10

# Printing input values
#print(a_cpu)
#print(b_cpu)
# Random function in Numpy always use float64
print(a_cpu.dtype)

c_cpu = grade2_vector(a_cpu, b_cpu, a, b, c)


# Evaluating the time

# Numba Python: huge improvement, better that numpy code
%timeit -n 5 -r 2 grade2_vector(a_cpu, b_cpu, a, b, c)

# w/ a numpy ufunc manually coded
%timeit -n 5 -r 2 grade2_ufunc(a_cpu, b_cpu, a, b, c)

# using the general numpy ufunc 
%timeit -n 5 -r 2 a*a_cpu**2 + b*b_cpu + c



float64
8.44 ms ± 97 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
18.4 ms ± 34.3 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
18.4 ms ± 27 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)


In [1]:
# ==============================
# Librerías
# ==============================
import numpy as np
import cupy as cp
from cupyx.profiler import benchmark

# ==============================
# Función para la operación
# ==============================
def grade2_ufunc_cupy(x, y, a, b, c):
    # x, y ya deben estar en GPU
    return a*x**2 + b*y + c

# ==============================
# Parámetros
# ==============================
size = 5_000_000
a_val = 3.5
b_val = 2.8
c_val = 10

# ==============================
# Caso 1: Copiando arrays desde CPU a GPU
# ==============================
# Arrays en CPU
a_cpu = np.random.rand(size)
b_cpu = np.random.rand(size)

# Copiamos a GPU
a_gpu = cp.asarray(a_cpu)
b_gpu = cp.asarray(b_cpu)

# Benchmark
t_copy = benchmark(grade2_ufunc_cupy, 
                   (a_gpu, b_gpu, a_val, b_val, c_val), 
                   n_repeat=5)
gpu_avg_time_copy = np.average(t_copy.gpu_times) * 1e3  # ms
print(f"Tiempo GPU (copiando arrays CPU→GPU): {gpu_avg_time_copy:.3f} ms")

# ==============================
# Caso 2: Arrays generados directamente en GPU
# ==============================
a_gpu_direct = cp.random.rand(size)
b_gpu_direct = cp.random.rand(size)

# Benchmark
t_direct = benchmark(grade2_ufunc_cupy, 
                     (a_gpu_direct, b_gpu_direct, a_val, b_val, c_val), 
                     n_repeat=5)
gpu_avg_time_direct = np.average(t_direct.gpu_times) * 1e3  # ms
print(f"Tiempo GPU (arrays generados directamente en GPU): {gpu_avg_time_direct:.3f} ms")


Tiempo GPU (copiando arrays CPU→GPU): 5.406 ms
Tiempo GPU (arrays generados directamente en GPU): 4.835 ms


In [3]:
# ==============================
# Librerías
# ==============================
import numpy as np
from numba import vectorize, float64
import cupy as cp
from cupyx.profiler import benchmark

# ==============================
# Función con Numba en GPU
# ==============================
@vectorize([float64(float64, float64, float64, float64, float64)], target='cuda')
def grade2_numba(x, y, a, b, c):
    return a*x**2 + b*y + c

# ==============================
# Parámetros
# ==============================
size = 5_000_000
a_val = 3.5
b_val = 2.8
c_val = 10

# ==============================
# Caso 1: Copia automática de CPU a GPU
# ==============================
# Arrays en CPU
a_cpu = np.random.rand(size)
b_cpu = np.random.rand(size)

# Benchmark (Numba copia automáticamente los arrays a GPU)
t_auto = benchmark(grade2_numba, (a_cpu, b_cpu, a_val, b_val, c_val), n_repeat=5)
gpu_avg_time_auto = np.average(t_auto.gpu_times) * 1e3
print(f"Tiempo GPU Numba (copia automática CPU→GPU): {gpu_avg_time_auto:.3f} ms")

# ==============================
# Caso 2: Arrays ya en GPU (sin contar la copia)
# ==============================
# Copiamos manualmente a GPU usando CuPy
a_gpu = cp.asarray(a_cpu)
b_gpu = cp.asarray(b_cpu)

# Convertimos los arrays de CuPy a Numba device array
from numba import cuda
a_device = cuda.as_cuda_array(a_gpu)
b_device = cuda.as_cuda_array(b_gpu)

# Benchmark con arrays ya en GPU
t_direct = benchmark(grade2_numba, (a_device, b_device, a_val, b_val, c_val), n_repeat=5)
gpu_avg_time_direct = np.average(t_direct.gpu_times) * 1e3
print(f"Tiempo GPU Numba (arrays ya en GPU): {gpu_avg_time_direct:.3f} ms")


Tiempo GPU Numba (copia automática CPU→GPU): 19.504 ms
Tiempo GPU Numba (arrays ya en GPU): 2.367 ms


### Explicación: 

Como podemos comprobar, el tiempo que se tarda es menor en si el array se crea directamente en la GPU que si lo pasamos de la CPU a la GPU.
Este incremento se debe al tiempo necesario para transferir los datos desde la memoria de la CPU a la GPU. Al generar los datos directamente en la GPU (por ejemplo, usando CuPy), se evita esta transferencia, lo que permite ahorrar ese tiempo.

Al comparar CuPy con Numba vemos que obtenemos un mayor rendimiento con esta última, sin embargo al pasar los arrays de CPU a GPu está mejor optimizado CuPy.

Observando los resultados podemos comprobar que el tiempo que se tarda en pasar un array de la memoria de la CPU a la GPU puede legar a formar un cuello de botella elevado.